In [2]:
import os
_dir = 'Employees by Assigned Position/'
path = [_dir + d for d in os.listdir(_dir)]
_dir2 = '12-Month Enrollment/'
path2 = [_dir2 + d for d in os.listdir(_dir2)]

In [4]:
import pandas as pd
EAP = pd.read_csv(path[0])
EAP_descp = pd.read_excel(path[1],engine='openpyxl')

## General-wise Employee per enrolled students

In [5]:
EFFY = pd.read_csv(path2[0])
EFFY_descp = pd.read_excel(path2[1],engine='openpyxl')

C:\Users\cb102\anaconda3\envs\ML\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
column_EAP = ['year','unitid','eapcat','facstat','eaptot']

In [34]:
df_EAP = EAP[column_EAP]
df_EAP = df_EAP[(df_EAP.facstat == 'All staff') & (df_EAP.eapcat == 'All staff')].reset_index(drop = True)

In [35]:
df_EAP.shape

(39605, 5)

In [32]:
df_EAP.head()

,year,unitid,facstat,eaptot
0,2020,100654,All staff,918
1,2020,100654,All staff,372
2,2020,100654,All staff,363
3,2020,100654,All staff,349
4,2020,100654,All staff,349


In [13]:
column_EFFY = ['year','unitid','effylev','efytotlt']

In [16]:
df_EFFY = EFFY[column_EFFY]
df_EFFY = df_EFFY[df_EFFY.effylev == 'All students total'].reset_index(drop = True)

In [17]:
df_EFFY.shape

(46146, 4)

In [25]:
enroll_school = set(df_EFFY.unitid)
stuff_school = set(df_EAP.unitid)
inters = enroll_school & stuff_school
print(f'Enroll: {len(enroll_school)}, stuff: {len(stuff_school)}, intersection: {len(inters)}')

Enroll: 7880, stuff: 7787, intersection: 7707


In [27]:
import numpy as np
print(np.unique(df_EFFY.year))
print(np.unique(df_EAP.year))

[2015 2016 2017 2018 2019 2020 2021]
[2015 2016 2017 2018 2019 2020]


In [36]:
df_join = pd.merge(df_EAP, df_EFFY, how = 'inner',left_on=['year','unitid'], right_on = ['year','unitid'])

In [42]:
df_eps = df_join[['year','unitid','eaptot','efytotlt']]
df_eps.columns = ['year','unitid','total_employee','enrolled_students']
df_eps['employee_per_student'] = df_eps.total_employee/df_eps.enrolled_students

C:\Users\cb102\anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
df_eps = df_eps[['year','unitid','employee_per_student']]

## Stuff-wise

In [46]:
np.unique(EAP.occupcat)

array(['All staff', 'Archivists, Curators, and Museum Technicians',
       'Business and Financial Operations',
       'Community Social Service, Legal, Arts, Design, Entertainment, Sports and Media',
       'Computer, Engineering, and Science',
       'Detailed library occupations not available for non-degree-granting institutions',
       'Graduate Assistants', 'Graduate Assistants  other',
       'Graduate Assistants Business and Financial Operations Occupations',
       'Graduate Assistants Community Social Service, Legal, Arts, Design, Entertainment, Sports and Media',
       'Graduate Assistants Computer, Engineering, and Science Occupations',
       'Graduate Assistants Healthcare Practioners and Technical',
       'Graduate Assistants Management Occupations',
       'Graduate Assistants Research', 'Graduate Assistants Teaching',
       'Graduate Assistants Total',
       'Healthcare Practioners and Technical',
       'Instructional staff - Instruction/research/public service',


In [82]:
df_EAP2 = EAP[column_EAP]
df_EAP2 = df_EAP2[(df_EAP2.eapcat == 'Research') | (df_EAP2.eapcat == 'All staff') | (df_EAP2.eapcat == 'Instructional staff, total')].reset_index(drop = True)

In [80]:
df_EAP2.head()

,year,unitid,eapcat,facstat,eaptot
0,2020,100654,All staff,All staff,918
1,2020,100654,Instructional staff - Instruction/research/pub...,All staff,14
2,2020,100654,Research,All staff,9
3,2020,100663,All staff,All staff,10093
4,2020,100663,Instructional staff - Instruction/research/pub...,All staff,2958


In [86]:
df_2 = pd.pivot_table(df_EAP2, values='eaptot', index=['year', 'unitid'],
                    columns=['eapcat'], aggfunc=np.sum).reset_index().fillna(0)
df_2['researcher_prop'] = df_2.Research/df_2['Instructional staff, total']
df_2['instructor_prop'] = df_2['Instructional staff, total']/df_2['All staff']
# df_2['researcher_prop'] = df_2.Research/df_2['All staff']
# df_2

In [88]:
df_2 = df_2[['year','unitid','All staff','researcher_prop','instructor_prop']]

In [89]:
df_2.shape

(39605, 5)

In [99]:
df_staff = pd.merge(df_eps,df_2,how = 'inner',left_on = ['year','unitid'], right_on = ['year','unitid'])

In [100]:
df_staff.shape

(39118, 6)

In [102]:
df_staff.to_csv('df_staff.csv',index = False)